## Put all imports necessary here

In [7]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import matplotlib.pyplot as plt

## Load the Data

In [8]:
# Load the training data
train_file = np.load('train.npz')
train_data = train_file['data']
print("train_data's shape", train_data.shape)

# Load the testing data
test_file = np.load('test_input.npz')
test_data = test_file['data']
print("test_data's shape", test_data.shape)


train_data's shape (10000, 50, 110, 6)
test_data's shape (2100, 50, 50, 6)


## Data Preprocessing

In [9]:
def prepare_data(data, is_train=True):
    """Extract ego vehicle's (agent 0) trajectories"""
    if is_train:
        X = data[:, 0, :50, :]  # (N, 50, 6) - First 50 timesteps
        Y = data[:, 0, 50:, :2]  # (N, 60, 2) - Next 60 (x,y)
    else:
        X = data[:, 0, :, :]  # (N, 50, 6) - Test uses only 50 timesteps
        Y = None
    return X, Y

X_train, Y_train = prepare_data(train_data)
X_test, _ = prepare_data(test_data, is_train=False)

# Normalize using training stats
pos_mean, pos_std = X_train[..., :2].mean(), X_train[..., :2].std()
X_train_norm = (X_train - pos_mean) / pos_std
Y_train_norm = (Y_train - pos_mean) / pos_std
X_test_norm = (X_test - pos_mean) / pos_std

# Convert to PyTorch tensors
train_dataset = TensorDataset(
    torch.FloatTensor(X_train_norm), 
    torch.FloatTensor(Y_train_norm)
)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

## Define Models

In [10]:
class TrajectoryMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.net = nn.Sequential(
            nn.Linear(50*6, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 60*2)  # Predict all 60 timesteps
        )
    
    def forward(self, x):
        x = self.flatten(x)
        return self.net(x).view(-1, 60, 2)  # Reshape to (batch, 60, 2)

model = TrajectoryMLP()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

## Do Training Loop

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(10):
    model.train()
    total_loss = 0
    for batch_X, batch_Y in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        batch_X, batch_Y = batch_X.to(device), batch_Y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_Y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Avg Loss: {total_loss/len(train_loader):.4f}")

Epoch 1: 100%|██████████| 157/157 [00:00<00:00, 361.02it/s]


Avg Loss: 0.0475


Epoch 2: 100%|██████████| 157/157 [00:00<00:00, 631.40it/s]


Avg Loss: 0.0056


Epoch 3: 100%|██████████| 157/157 [00:00<00:00, 654.83it/s]


...

Avg Loss: 0.0042


Epoch 9: 100%|██████████| 157/157 [00:00<00:00, 606.49it/s]


Avg Loss: 0.0043


Epoch 10: 100%|██████████| 157/157 [00:00<00:00, 600.89it/s]

Avg Loss: 0.0044


## Predict and Create Submission

In [12]:
model.eval()
with torch.no_grad():
    test_input = torch.FloatTensor(X_test_norm).to(device)
    preds_norm = model(test_input).cpu().numpy()  # (2100, 60, 2)

# Denormalize
preds = preds_norm * pos_std + pos_mean

# Create submission
submission = preds.reshape(-1, 2)  # Flatten to (2100*60, 2)
submission_df = pd.DataFrame(submission, columns=['x', 'y'])
submission_df.index.name = 'index'
submission_df.to_csv('submission.csv')

print("Submission saved! Example predictions:")
print(submission_df.head())

Submission saved! Example predictions:
                 x            y
index                          
0      5446.548828  1525.442505
1      5451.425293  1518.542847
2      5454.622070  1513.448730
3      5440.516602  1511.121094
4      5441.444824  1482.686523
